# Map georeferencing notebook - Python

In [1]:
import copy
import datetime
import io
import locale
import os
import folium
import pandas as pd

from geopy.geocoders import Nominatim
from matplotlib import colors as mcolors

# from folium.plugins import MarkerCluster
# from folium import Choropleth, Circle, Marker

# TODO
descrizione metodo

In [2]:
def SearchNewPlace(place_to_find = "Italy"):
    geolocator = Nominatim(user_agent="UniMolFinalExam")
    geo_cordinates = geolocator.geocode(place_to_find)
    # print(geo_cordinates)
    return geo_cordinates

# TODO
descrizione metodo

Questo metodo elimina dal corrente dataframe tutte le colonne, eccezion fatta per latitudine e longitudine

In [3]:
def CleanHeader(file_to_process):

    dataTypeDict = {"GPS Time":str ,"Device Time":str ,"Longitude":float, "Latitude":float, "GPS Speed (Meters/second)":float, "Altitude":float, "G(x)":float, "G(y)":float, "G(z)":float, "Kilometers Per Litre(Instant)(kpl)": float, "Litres Per 100 Kilometer(Instant)(l/100km)":float, "Actual engine % torque(%)":float, "Trip Distance(km)":float, "Engine RPM(rpm)":float, "GPS Accuracy(m)":float, "GPS Altitude(m)":float, "GPS Latitude(°)":float, "GPS Longitude(°)":float, "Acceleration Sensor(X axis)(g)":float, "Acceleration Sensor(Y axis)(g)":float, "Acceleration Sensor(Z axis)(g)":float, "Acceleration Sensor(Total)(g)":float, "Run time since engine start(s)":int, "Driver":str}

    df = pd.read_csv(file_to_process, dtype=dataTypeDict)
    # keeps_only_latidue_longitude = [ "GPS Time","Device Time","GPS Speed (Meters/second)","Altitude","G(x)","G(y)","G(z)","Kilometers Per Litre(Instant)(kpl)","Litres Per 100 Kilometer(Instant)(l/100km)","Actual engine % torque(%)","Trip Distance(km)","Engine RPM(rpm)","GPS Accuracy(m)","GPS Altitude(m)","GPS Latitude(°)","GPS Longitude(°)","Acceleration Sensor(X axis)(g)","Acceleration Sensor(Y axis)(g)","Acceleration Sensor(Z axis)(g)","Acceleration Sensor(Total)(g)","Run time since engine start(s)","Driver"]
    keeps_only_latidue_longitude_time = [ "GPS Time","GPS Speed (Meters/second)","Altitude","G(x)","G(y)","G(z)","Kilometers Per Litre(Instant)(kpl)","Litres Per 100 Kilometer(Instant)(l/100km)","Actual engine % torque(%)","Trip Distance(km)","Engine RPM(rpm)","GPS Accuracy(m)","GPS Altitude(m)","GPS Latitude(°)","GPS Longitude(°)","Acceleration Sensor(X axis)(g)","Acceleration Sensor(Y axis)(g)","Acceleration Sensor(Z axis)(g)","Acceleration Sensor(Total)(g)","Run time since engine start(s)","Driver"]

    # gps_long_lat = df.drop(columns=keeps_only_latidue_longitude)    
    # return gps_long_lat

    gps_long_lat_time = df.drop(columns=keeps_only_latidue_longitude_time)
    return gps_long_lat_time

# TODO
descrizione metodo

In [4]:
def AddCsvToDf(file_to_process):
    return pd.read_csv(file_to_process)

# TODO
descrizione metodo

In [5]:
def ParseGpsTime(gps_value:str):
    local_setting_gps_time = 'en_US'
    locale.setlocale(locale.LC_ALL, local_setting_gps_time)

    gps_format = '%a %b %d %H:%M:%S %z %Y'
    # gps_test = "Mon Dec 30 09:16:50 +0100 2019"

    date_time_gps = datetime.datetime.strptime(gps_value, gps_format)

    # print("GPS:")
    # print('Date:', date_time_gps.date())
    # print('Time:', date_time_gps.time())
    # print('Date-time:', date_time_gps)

    return date_time_gps

codice per la gestione delle time stanp nel csv

In [6]:
def ParseDeviceTime(device_value:str):
    local_setting_device_time = 'it_IT'
    locale.setlocale(locale.LC_ALL, local_setting_device_time)

    device_format = '%d-%b-%Y %H:%M:%S.%f'
    # device_time = "30-dic-2019 09:16:52.347"

    date_time_device = datetime.datetime.strptime(device_value, device_format)

    # print("Device:")
    # print('Date:', date_time_device.date())
    # print('Time:', date_time_device.time())
    # print('Date-time:', date_time_device)

    return date_time_device

# TODO
descrizione metodo

In [7]:
def CreateNewMap(latitude = None, longitude = None, map_zoom = 7):
    if ((latitude == None) and (longitude == None)):
        default_location = SearchNewPlace()
        return folium.Map(location=[default_location.latitude,default_location.longitude], zoom_start=map_zoom)
    else:
        return folium.Map(location=[latitude,longitude], zoom_start=map_zoom)

aggiunte i punti sulla mappa

In [8]:
def SaveMap(map, driver_label, date = None, note = ""):
    extension = "html"
    if (date == None):
        file_name = "{}Driver{}_full_map.{}".format(note, driver_label, extension)
    else:
        file_name = "{}Driver_{}_{}_map.{}".format(note, driver_label, date, extension)

    map.save(file_name)

    print(file_name)

In [9]:
def CreatePointOnMap(map, row, user_label, time_stamp, col, rad = 20):
    folium.Circle(
        location = [row["Latitude"], row["Longitude"]],
        radius = rad,
        popup = "Driver: {} | Date: {} | Time: {}".format(
            user_label, time_stamp.date(), time_stamp.time()
            ),
        color = col,
        fill = True,
        fill_color = col
    ).add_to(map)

In [10]:
def CreateLineOnMap(map, points, user_label, time_stamp, col):
    folium.PolyLine(
        locations = points,
        tooltip = "Driver: {} | Date: {}".format(
            user_label, time_stamp.date()
            ),
        color = col
    ).add_to(map)

# TODO
descrizione metodo

In [11]:
def Add_Points_To_Unique_Map_Export(users_list, dataframe):
    map_dictionary = dict()

    color_code = 1

    for user in users_list:
        current_map = None
        current_map = CreateNewMap()

        color_list = None
        color_list = copy.deepcopy(mcolors.CSS4_COLORS)

        current_color = color_list.popitem()

        for index in range(len(dataframe[user])):

            current_row = dataframe[user].iloc[index]
            
            actual_device_time = ParseDeviceTime(
                current_row["Device Time"])

            if (index > 0):
                previous_row = dataframe[user].iloc[index-1]
                previous_device_time = ParseDeviceTime(
                    previous_row["Device Time"])

                if(previous_device_time.date() != actual_device_time.date()):
                    # se giorni diversi cambia colore cambia colore
                    current_color = color_list.popitem()

            CreatePointOnMap(current_map, current_row, user, actual_device_time, current_color[color_code])

        map_dictionary[user] = current_map
        SaveMap(map=map_dictionary[user], driver_label=user, note="Points_")

    # return map_dictionary

# TODO
descrizione metodo
all'interno della singola mappa, il colore del percorso sarà sempre lo stesso

In [12]:
def Add_Points_To_Different_Maps(users_list, dataframe):

    current_color = mcolors.CSS4_COLORS["red"]

    for user in users_list:
        current_map = None
        current_map = CreateNewMap()
        
        for index in range(len(dataframe[user])):

            current_row = dataframe[user].iloc[index]
            
            actual_device_time = ParseDeviceTime(current_row["Device Time"])

            if (index > 0):
                previous_row = dataframe[user].iloc[index-1]
                previous_device_time = ParseDeviceTime(previous_row["Device Time"])

                if(previous_device_time.date() != actual_device_time.date()):
                    SaveMap(date = actual_device_time.date(), driver_label = user, map = current_map)
                    current_map = CreateNewMap()
            
            CreatePointOnMap(current_map, current_row, user, actual_device_time, current_color)

In [13]:
def Add_Path_To_Unique_Map_Export(users_list, dataframe):

    map_dictionary = dict()

    color_code = 1

    for user in users_list:
        current_map = None
        current_map = CreateNewMap()

        color_list = None
        color_list = copy.deepcopy(mcolors.CSS4_COLORS)

        current_color = color_list.popitem()

        current_points = list()

        for index in range(len(dataframe[user])):

            current_row = dataframe[user].iloc[index]
            
            actual_device_time = ParseDeviceTime(current_row["Device Time"])
            
            current_points.append(tuple([current_row["Latitude"], current_row["Longitude"]]))

            if (index > 0):
                previous_row = dataframe[user].iloc[index-1]
                previous_device_time = ParseDeviceTime(previous_row["Device Time"])

                if(previous_device_time.date() != actual_device_time.date()):
                    CreateLineOnMap(map=current_map, points=current_points, user_label=user, time_stamp=actual_device_time, col=current_color[color_code])
                    current_color = color_list.popitem()
                    current_points = list()

        map_dictionary[user] = current_map
        SaveMap(map=map_dictionary[user], driver_label=user, note="Path_")

    # return map_dictionary

# Inizio codice

In [14]:
# urllib.request.urlretrieve(url, filename=file_name)

In [15]:
# url = "https://raw.githubusercontent.com/AngelusGi/UniMolFinalExam/main/Dataset/Original_CSV/Angelo/trackLog-2019-dic-23_13-53-06.csv"
# file_name = "trackLog-2019-dic-23_13-53-06.csv"

prede tutti i file nella cartella e li trasforma in un dizionario di dataframe

In [16]:
basepath = os.environ['USERPROFILE']

local_user = "angel"
vm_user = "vmuser"

if local_user in basepath:
    githubpath = r"\GitHub\UniMolFinalExam"
    current_env = local_user
elif vm_user in basepath:
    githubpath = r"\Desktop\GitHub\UniMolFinalExam"
    current_env = vm_user

In [17]:
# csv_path = r"{}{}\Notebooks\Python\TestSet".format(basepath,githubpath)
csv_path = r"{}{}\Dataset\Final".format(basepath,githubpath)

In [18]:
coordinates_data = dict()
full_data = dict()

In [19]:
for directory, subdir_list, file_list in os.walk(csv_path):
    for file_name in file_list:
        print('File:', file_name)
        path_to_read_csv = csv_path + "\\" + file_name
        print(path_to_read_csv)

        temp_dataframe = CleanHeader(path_to_read_csv)
        temp_file_name_ext = file_name.replace('.csv','')
        final_file_name = temp_file_name_ext.replace('trackLog-','')
        coordinates_data[final_file_name] = temp_dataframe

        full_data[final_file_name] = AddCsvToDf(path_to_read_csv)
        print()
    print()

File: A.csv
C:\Users\vmuser\Desktop\GitHub\UniMolFinalExam\Dataset\Final\A.csv

File: B.csv
C:\Users\vmuser\Desktop\GitHub\UniMolFinalExam\Dataset\Final\B.csv




crea un dizionario contenete solo le coordinate di latitudine e longitudine

In [20]:
coordinates_data

{'A':                     Device Time  Longitude   Latitude
 0      23-dic-2019 13:53:09.763  13.931077  42.048168
 1      23-dic-2019 13:53:12.432  13.931068  42.048161
 2      23-dic-2019 13:53:15.081  13.931075  42.048154
 3      23-dic-2019 13:53:17.284  13.931077  42.048154
 4      23-dic-2019 13:53:19.504  13.931077  42.048155
 ...                         ...        ...        ...
 74438  01-gen-2020 13:19:56.110  13.931075  42.048150
 74439  01-gen-2020 13:19:57.318  13.931076  42.048151
 74440  01-gen-2020 13:19:58.529  13.931075  42.048142
 74441  01-gen-2020 13:19:59.743  13.931065  42.048130
 74442  01-gen-2020 13:20:00.949  13.931066  42.048119
 
 [74443 rows x 3 columns],
 'B':                      Device Time  Longitude   Latitude
 0       07-dic-2019 18:55:34.543  13.926052  42.050242
 1       07-dic-2019 18:55:35.276  13.926052  42.050242
 2       07-dic-2019 18:55:35.430  13.926052  42.050242
 3       07-dic-2019 18:55:38.522  13.926060  42.050236
 4       07-dic-2019 

In [21]:
drivers_list = ["A", "B"]

for driver in drivers_list:
    print("Driver -> {}".format(driver))

    print("Info ->")
    print()
    print(coordinates_data[driver].info())
    print()
    
    print("Head ->")
    print()
    print(coordinates_data[driver].head())
    print()
    print("##########################################")
    print()

Driver -> A
Info ->

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74443 entries, 0 to 74442
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Device Time  74443 non-null  object 
 1   Longitude    74443 non-null  float64
 2   Latitude     74443 non-null  float64
dtypes: float64(2), object(1)
memory usage: 1.7+ MB
None

Head ->

                Device Time  Longitude   Latitude
0  23-dic-2019 13:53:09.763  13.931077  42.048168
1  23-dic-2019 13:53:12.432  13.931068  42.048161
2  23-dic-2019 13:53:15.081  13.931075  42.048154
3  23-dic-2019 13:53:17.284  13.931077  42.048154
4  23-dic-2019 13:53:19.504  13.931077  42.048155

##########################################

Driver -> B
Info ->

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188795 entries, 0 to 188794
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Device Time  188795 non-nul

estrae un singolo dataframe

In [22]:
gps_long_lat_B = coordinates_data["B"]
gps_long_lat_A = coordinates_data["A"]

device_timeA = ParseDeviceTime(gps_long_lat_A["Device Time"][0])
device_timeB = ParseDeviceTime(gps_long_lat_B["Device Time"][0])

In [23]:
print("A : {}".format(device_timeA))
print("B : {}".format(device_timeB))
device_timeA.date() == device_timeB.date()

A : 2019-12-23 13:53:09.763000
B : 2019-12-07 18:55:34.543000


False

In [24]:
first_device_time = ParseDeviceTime(gps_long_lat_A["Device Time"][0])
second_device_time = ParseDeviceTime(gps_long_lat_A["Device Time"][1])
third_device_time = ParseDeviceTime(gps_long_lat_A["Device Time"][17248])
fourth_device_time = ParseDeviceTime(gps_long_lat_A["Device Time"][17249])

In [25]:
# test Times from the same day
print("A 1st : {}".format(first_device_time))
print("A 2nd : {}".format(second_device_time))
first_device_time.date() == second_device_time.date()

A 1st : 2019-12-23 13:53:09.763000
A 2nd : 2019-12-23 13:53:12.432000


True

In [26]:
# test Times from different days
print("A 1st : {}".format(first_device_time))
print("A 3nd : {}".format(third_device_time))
first_device_time.date() == third_device_time.date()

A 1st : 2019-12-23 13:53:09.763000
A 3nd : 2019-12-30 23:58:26.738000


False

In [27]:
# test Times from different days (close days/time)
print("A 3nd : {}".format(third_device_time))
print("A 4nd : {}".format(fourth_device_time))
third_device_time.date() == fourth_device_time.date()

A 3nd : 2019-12-30 23:58:26.738000
A 4nd : 2019-12-31 00:01:41.749000


False

ottenere un indirizzo a partire dalle coordinate

In [28]:
geolocator = Nominatim(user_agent="UniMolFinalExam")

In [29]:
italy_coordinates = SearchNewPlace()

In [30]:
italy_coordinates

Location(Italia, (42.6384261, 12.674297, 0.0))

In [31]:
# italy_map = CreateNewMap(italy_coordinates.latitude, italy_coordinates.longitude, map_zoom=6)
italy_map = CreateNewMap(italy_coordinates.latitude, italy_coordinates.longitude)

In [32]:
italy_map

In [33]:
coordinates_data

{'A':                     Device Time  Longitude   Latitude
 0      23-dic-2019 13:53:09.763  13.931077  42.048168
 1      23-dic-2019 13:53:12.432  13.931068  42.048161
 2      23-dic-2019 13:53:15.081  13.931075  42.048154
 3      23-dic-2019 13:53:17.284  13.931077  42.048154
 4      23-dic-2019 13:53:19.504  13.931077  42.048155
 ...                         ...        ...        ...
 74438  01-gen-2020 13:19:56.110  13.931075  42.048150
 74439  01-gen-2020 13:19:57.318  13.931076  42.048151
 74440  01-gen-2020 13:19:58.529  13.931075  42.048142
 74441  01-gen-2020 13:19:59.743  13.931065  42.048130
 74442  01-gen-2020 13:20:00.949  13.931066  42.048119
 
 [74443 rows x 3 columns],
 'B':                      Device Time  Longitude   Latitude
 0       07-dic-2019 18:55:34.543  13.926052  42.050242
 1       07-dic-2019 18:55:35.276  13.926052  42.050242
 2       07-dic-2019 18:55:35.430  13.926052  42.050242
 3       07-dic-2019 18:55:38.522  13.926060  42.050236
 4       07-dic-2019 

In [34]:
Add_Path_To_Unique_Map_Export(users_list=drivers_list, dataframe=coordinates_data)

Path_DriverA_full_map.html
Path_DriverB_full_map.html


In [35]:
Add_Points_To_Different_Maps(users_list=drivers_list, dataframe=coordinates_data)

Driver_A_2019-12-26_map.html
Driver_A_2019-12-30_map.html
Driver_A_2019-12-31_map.html
Driver_A_2019-11-02_map.html
Driver_A_2019-11-03_map.html
Driver_A_2019-10-03_map.html
Driver_A_2019-10-06_map.html
Driver_A_2019-10-07_map.html
Driver_A_2019-10-08_map.html
Driver_A_2019-10-09_map.html
Driver_A_2019-10-10_map.html
Driver_A_2019-10-13_map.html
Driver_A_2019-10-14_map.html
Driver_A_2019-09-23_map.html
Driver_A_2019-09-24_map.html
Driver_A_2019-09-26_map.html
Driver_A_2019-09-27_map.html
Driver_A_2019-09-28_map.html
Driver_A_2019-09-29_map.html
Driver_A_2020-01-01_map.html
Driver_B_2019-12-08_map.html
Driver_B_2019-12-09_map.html
Driver_B_2019-12-11_map.html
Driver_B_2019-12-12_map.html
Driver_B_2019-12-13_map.html
Driver_B_2019-12-14_map.html
Driver_B_2019-12-16_map.html
Driver_B_2019-12-17_map.html
Driver_B_2019-12-18_map.html
Driver_B_2019-12-19_map.html
Driver_B_2019-12-20_map.html
Driver_B_2019-11-03_map.html
Driver_B_2019-11-04_map.html
Driver_B_2019-11-20_map.html
Driver_B_2019-

In [36]:
Add_Points_To_Unique_Map_Export(users_list=drivers_list, dataframe=coordinates_data)

Points_DriverA_full_map.html
Points_DriverB_full_map.html
